# Mendeley Sync

Script ini melakukan sinkronisasi data peneliti dari [Mendeley](http://www.mendeley.com) ke website lab berbasis [Yoopa](http://yoopa.readthedocs.io).

## Prasyarat

1. Mendaftarkan App di [Mendeley Developer](http://dev.mendeley.com) dan mendapat API secret.
   Untuk http://www.lskk.org, App ID-nya adalah 3301 (dapatkan App Secret dari admin yang berwenang).
2. Menginstall [Mendeley Python SDK](https://github.com/Mendeley/mendeley-python-sdk), caranya:

        pip install mendeley

3. Dapatkan access token dan refresh access token dari https://mendeley-show-me-access-tokens.herokuapp.com/
4. Set _environment variables_ `MENDELEY_CLIENT_ID`, `MENDELEY_CLIENT_SECRET`, `MENDELEY_REDIRECT_URI`, `MENDELEY_ACCESS_TOKEN`, dan `MENDELEY_REFRESH_TOKEN` sebelum menjalankan Jupyter/IPython.

        set MENDELEY_CLIENT_ID=3301
        set MENDELEY_CLIENT_SECRET=4n...
        set MENDELEY_REDIRECT_URI=https://www.lskk.org/
        set MENDELEY_ACCESS_TOKEN=MS...eXc
        set MENDELEY_REFRESH_TOKEN=MS...TlE

## Mendapatkan Access Token dan Refresh Token Baru

Bila belum memiliki access token, Anda harus mendapatkan token baru. Login dulu ke akun Mendeley Anda. Lalu jalankan script berikut:

In [37]:
import os
from mendeley import Mendeley

# These values should match the ones supplied when registering your application.
mendeley = Mendeley(os.environ['MENDELEY_CLIENT_ID'],
                    client_secret=os.environ['MENDELEY_CLIENT_SECRET'],
                    redirect_uri=os.environ['MENDELEY_REDIRECT_URI'])

auth = mendeley.start_authorization_code_flow()

# The user needs to visit this URL, and log in to Mendeley.
login_url = auth.get_login_url()
print('Silakan buka browser: %s' % login_url)

Silakan buka browser: https://api.mendeley.com/oauth/authorize?response_type=code&client_id=3301&redirect_uri=https%3A%2F%2Fwww.lskk.org%2F&scope=all&state=IRZIPVC7M0MGFXAMFPMXZBOSKH6ROK


Buka alamat yang ditampilkan di atas, maka Anda akan di-redirect ke URL tertentu, masukkan URL tersebut di variabel `auth_response` berikut, lalu jalankan scriptnya untuk mendapatkan `session` bertipe [`MendeleySession`](https://mendeley-python.readthedocs.io/en/latest/usage.html#mendeley.session.MendeleySession).

In [ ]:
# After logging in, the user will be redirected to a URL, auth_response.
auth_response = 'https://www.lskk.org/?code=G-73jbo5ezhiJquSzP44M-5aoFU&state=HT6DVX7XOSS3O02CJ5MERWFUZX22CG'
session = auth.authenticate(auth_response)

Anda dapat menampilkan isi `session.token`. Token tersebut dapat disimpan ke `MENDELEY_ACCESS_TOKEN` dan `MENDELEY_REFRESH_TOKEN`.

In [ ]:
import pprint
pprint.pprint(session.token)

## Membuat MendeleySession dari access token yang sudah ada

Bila `MENDELEY_ACCESS_TOKEN` dan `MENDELEY_REFRESH_TOKEN` sudah disimpan, maka kita dapat langsung menggunakannya:

In [25]:
import os
from mendeley.session import MendeleySession
from mendeley.auth import MendeleyAuthorizationCodeTokenRefresher

token = {'access_token': os.environ['MENDELEY_ACCESS_TOKEN'],
 'expires_at': 1467734155.9604628,
 'expires_in': 3600,
 'refresh_token': os.environ['MENDELEY_REFRESH_TOKEN'],
 'scope': ['all'],
 'token_type': 'bearer'}
auth = mendeley.start_authorization_code_flow()
session = MendeleySession(mendeley, token, client=auth.client, refresher=MendeleyAuthorizationCodeTokenRefresher(auth))
session

## Mengakses Documents

Dapatkan _documents_ dari user tersebut sebagai [`mendeley.pagination.Page`](https://mendeley-python.readthedocs.io/en/latest/usage.html#mendeley.pagination.Page):

In [35]:
docs = session.documents.list(view='all', page_size=20)
pprint.pprint(list(map(lambda d: (d.id, d.title, d.authored, d.json['authors']), docs.items)))

[('702d88e7-321f-3da4-bd9d-2120d9c46044',
  'Implementation of graph database for OpenCog artificial general '
  'intelligence framework using Neo4j',
  True,
  [{'first_name': 'Hendy', 'last_name': 'Irawan'},
   {'first_name': 'Ary Setijadi', 'last_name': 'Prihatmanto'}]),
 ('3dc8b0c2-917c-3ca8-8eed-09a5072230e4',
  'Design and Implementation of Image Processing System for Lumen Social '
  'Robot-Humanoid as an Exhibition Guide for Electrical Engineering Days',
  False,
  [{'first_name': 'Setyaki Sholata', 'last_name': 'Sya'},
   {'first_name': 'Ary Setijadi', 'last_name': 'Prihatmanto'}]),
 ('77bdcaa1-5b77-3438-97ce-4a7f637a16de',
  'Artificial Intelligence Platform and Graph-Based Semantic Reasoning with '
  'Natural Language User Interface for Lumen Robot Friend',
  True,
  [{'first_name': 'Hendy', 'last_name': 'Irawan'},
   {'first_name': 'Ary Setijadi', 'last_name': 'Prihatmanto'}]),
 ('400aba71-eddc-382e-9b17-3918657de4a6',
  'Design and Implementation of Audio Communication Sys

Setiap _document_ berupa [`UserDocument`](https://mendeley-python.readthedocs.io/en/latest/models.html#mendeley.models.documents.UserDocument) yang dapat kita tampilkan:

In [36]:
for doc in docs.items:
    pprint.pprint(doc.json)

{'abstract': 'We create a graph backing store for OpenCog using Neo4j graph '
             'database. The GraphBackingStore API extends the current '
             'BackingStore C++ API. It can take special queries that map '
             'naturally into Cypher queries and simple manipulations of Neo4j '
             'graph traversals. The Neo4j node-relationship structures and '
             'custom indices are optimized for AtomSpace usage and '
             'performance, with considerations for integration with Linked '
             'Data. This makes it easier to integrate projects which use '
             'common Linked Data foundation, such as Lumen Robot Friend. Data '
             'access is using Protocol Buffers-based protocol over ZeroMQ '
             'messaging transport. The initial work for Neo4j Graph Backing '
             'Store for OpenCog was implemented during Google Summer of Code '
             '2015. All source code is open source and available at '
             '

Contoh struktur _document_ dalam JSON sebagai berikut:
    
    {'abstract': 'We create a graph backing store for OpenCog using Neo4j graph '
                 'database. The GraphBackingStore API extends the current '
                 'BackingStore C++ API. It can take special queries that map '
                 'naturally into Cypher queries and simple manipulations of Neo4j '
                 'graph traversals. The Neo4j node-relationship structures and '
                 'custom indices are optimized for AtomSpace usage and '
                 'performance, with considerations for integration with Linked '
                 'Data. This makes it easier to integrate projects which use '
                 'common Linked Data foundation, such as Lumen Robot Friend. Data '
                 'access is using Protocol Buffers-based protocol over ZeroMQ '
                 'messaging transport. The initial work for Neo4j Graph Backing '
                 'Store for OpenCog was implemented during Google Summer of Code '
                 '2015. All source code is open source and available at '
                 'https://github.com/ceefour/opencog- neo4j, '
                 'https://github.com/opencog/atomspace, '
                 'andhttps://github.com/opencog/opencog. languages. The AtomSpace '
                 'accesses the Neo4j Backing Store via a ZeroMQ Backing Store '
                 'implementation, which implements the AtomSpace C++ BackingStore '
                 'API using message patterns with Protocol Buffers serialization '
                 'and transported via ZeroMQ.',
     'authored': True,
     'authors': [{'first_name': 'Hendy', 'last_name': 'Irawan'},
                 {'first_name': 'Ary Setijadi', 'last_name': 'Prihatmanto'}],
     'confirmed': True,
     'created': '2016-05-28T02:05:42.000Z',
     'file_attached': True,
     'hidden': False,
     'id': '702d88e7-321f-3da4-bd9d-2120d9c46044',
     'keywords': ['artificial',
                  'graph database',
                  'intelligence',
                  'knowledge representation',
                  'languages',
                  'linked data',
                  'neo4j',
                  'neo4j backing store',
                  'opencog',
                  'protocol buffers',
                  'semantic web',
                  'the atomspace accesses the',
                  'yago',
                  'zeromq'],
     'last_modified': '2016-06-02T11:16:34.000Z',
     'profile_id': 'f9c445a9-0722-3163-99b2-3d559c9b9e59',
     'read': False,
     'source': '4th International Conference on Interactive Digital Media',
     'starred': False,
     'title': 'Implementation of graph database for OpenCog artificial general '
              'intelligence framework using Neo4j',
     'type': 'conference_proceedings',
     'year': 2015}    

## Mendapatkan Daftar Publikasi dari Yoopa/Drupal

Untuk mendapatkan sebuah _user_ beserta anak _content_ (terutama _publications_), kita dapat memanggil REST service misalnya http://lskk.org.amanahwin/user/1?_format=hal_json (harus diprotect permission karena dapat berisi informasi sensitif).

Contoh hasilnya:

    {
      "_links": {
        "self": {
          "href": "http://lskk.org.amanahwin/user/1?_format=hal_json"
        },
        "type": {
          "href": "http://lskk.org.amanahwin/rest/type/user/user"
        },
        "http://lskk.org.amanahwin/rest/relation/user/user/field_publications": [
          {
            "href": "http://lskk.org.amanahwin/node/17?_format=hal_json"
          }
        ],
        "http://lskk.org.amanahwin/rest/relation/user/user/user_picture": [
          {
            "href": "http://lskk.org.amanahwin/sites/default/files/pictures/2016-07/Ary_Prihatmanto.jpg",
            "lang": "en"
          }
        ]
      },
      "uid": [
        {
          "value": "1"
        }
      ],
      "uuid": [
        {
          "value": "bf62b817-b5c7-481e-9e66-472b3bcc9cc1"
        }
      ],
      "langcode": [
        {
          "value": "en",
          "lang": "en"
        }
      ],
      "preferred_langcode": [
        {
          "value": "en"
        }
      ],
      "name": [
        {
          "value": "asetijadi"
        }
      ],
      "mail": [
        {
          "value": "as***@lskk.ee.itb.ac.id"
        }
      ],
      "timezone": [
        {
          "value": "Asia/Jakarta"
        }
      ],
      "status": [
        {
          "value": "1"
        }
      ],
      "created": [
        {
          "value": "1467656214"
        }
      ],
      "changed": [
        {
          "value": "1467915667",
          "lang": "en"
        }
      ],
      "access": [
        {
          "value": "1467917878"
        }
      ],
      "login": [
        {
          "value": "1467656458"
        }
      ],
      "init": [
        {
          "value": "as***@lskk.ee.itb.ac.id"
        }
      ],
      "roles": [
        {
          "target_id": "administrator"
        }
      ],
      "default_langcode": [
        {
          "value": "1",
          "lang": "en"
        }
      ],
      "field_name": [
        {
          "value": "Ary Setijadi Prihatmanto"
        }
      ],
      "_embedded": {
        "http://lskk.org.amanahwin/rest/relation/user/user/field_publications": [
          {
            "_links": {
              "self": {
                "href": "http://lskk.org.amanahwin/node/17?_format=hal_json"
              },
              "type": {
                "href": "http://lskk.org.amanahwin/rest/type/node/publication"
              }
            },
            "uuid": [
              {
                "value": "9db34be9-0750-44e5-88eb-d2a95ea26b37"
              }
            ]
          }
        ],
        "http://lskk.org.amanahwin/rest/relation/user/user/user_picture": [
          {
            "_links": {
              "self": {
                "href": "http://lskk.org.amanahwin/sites/default/files/pictures/2016-07/Ary_Prihatmanto.jpg"
              },
              "type": {
                "href": "http://lskk.org.amanahwin/rest/type/file/file"
              }
            },
            "uuid": [
              {
                "value": "c5fab6c4-8197-4626-9c8a-5ad0090105e6"
              }
            ],
            "uri": [
              {
                "value": "http://lskk.org.amanahwin/sites/default/files/pictures/2016-07/Ary_Prihatmanto.jpg"
              }
            ],
            "lang": "en"
          }
        ]
      }
    }
    
Namun cara tersebut kurang efisien karena hanya mendapatkan URI ke publikasi. Padahal kita ingin mendapatkan data publikasi juga, terutama title, authors, dan type.

## Mendapatkan data semua publications dari Yoopa/Drupal

Dengan jumlah publikasi terbatas (< 10.000) dan akses intranet, maka kita dapat menyimpan metadata semua publikasi di memory Python untuk nantinya dicocokkan dengan data yang didapatkan dari Mendeley.

TODO: Implement